In [1]:
import fxcmpy
import pandas as pd
import datetime as dt
import requests
import configparser
import zipfile
import os
import pandas

from tqdm import tqdm

Can not find FXCM Server.

When the market is not online.

In [ ]:
con = fxcmpy.fxcmpy(config_file='C:\\Users\\Dema\\Documents\\Programming\\Python\\Projects\\FX\\configs\\api.cfg')

### Price Data

In [95]:
all_instruments = con.get_instruments()

freq_dict = {'m5':'5min', 'm15': '15min', 'm30': '30min', 'H1': '1H', 'H4': '4H', 'D1': '1D', 'W1': '1W'}

first_date = pd.Timestamp('1990-01-01')
last_date = pd.Timestamp('2022-01-01')

all_pairs = ['GBP/USD', 'USD/JPY', 'USD/CHF', 'USD/CAD',
             'AUD/USD', 'AUD/CAD', 'AUD/CHF', 'AUD/JPY',
             'CAD/CHF', 'CAD/JPY', 'CHF/JPY', 'GBP/AUD',
             'GBP/CAD', 'GBP/CHF', 'GBP/JPY', 'VOLX',
             'USEquities', 'USDOLLAR', 'UK100']

In [96]:
def get_instrument_data(instrument, frequency, first_date, last_date):
    
    dates = list(pd.date_range(start=first_date, end=last_date, freq=freq_dict[frequency])[::10000]) + [last_date]
    
    data = [con.get_candles(instrument, period=frequency, start=dates[ix], stop=dates[ix+1]) for ix in range(len(dates)-1)]
    
    return pd.concat(data).drop_duplicates()

def get_and_save(all_pairs, frequency, first_date, last_date):
    
    all_data = {pair: get_instrument_data(pair, frequency, first_date, last_date) for pair in tqdm(all_pairs)}
    
    print(f'Saving {frequency} data.')
    
    for pair, data in all_data.items():
        pair_name = pair.replace('/', '')
        data.to_csv(f'data/raw/prices/{frequency}/{pair_name}.csv')

In [97]:
get_and_save(all_pairs, "W1", first_date, last_date)
get_and_save(all_pairs, "D1", first_date, last_date)
get_and_save(all_pairs, "H4", first_date, last_date)
get_and_save(all_pairs, "H1", first_date, last_date)
get_and_save(all_pairs, "m30", first_date, last_date)
get_and_save(all_pairs, "m15", first_date, last_date)
get_and_save(all_pairs, "m5", first_date, last_date)


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:04<00:00,  4.32it/s]

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

Saving W1 data.



100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:10<00:00,  1.89it/s]


Saving D1 data.



100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:26<00:00,  4.58s/it]


Saving H4 data.



100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [03:14<00:00, 10.22s/it]


Saving H1 data.


### COT data

In [100]:
# Only downloading the latest - will have to change the link and year on new years
path = 'data/raw/cot/zip/com_fin_xls_2020.zip'

config = configparser.RawConfigParser()
config.read('configs/cot.cfg')
cot_dict = dict(config.items('DOWNLOAD'))

zip_url = cot_dict['zip_url']

r = requests.get(zip_url, stream=True)
with open(path, 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk) 

In [101]:
os.remove('data/raw/cot/xls/2020.xls')

with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall('data/raw/cot/xls/')
    
os.rename(r'data/raw/cot/xls/FinComYY.xls', r'data/raw/cot/xls/2020.xls')

In [102]:
xls_path = 'data/raw/cot/xls/'
all_files = os.listdir(xls_path)
all_paths = {file.strip('.xls'): xls_path + file for file in all_files}

for file_name, path in all_paths.items():
    xls_file = pd.read_excel(path, sheet_name='XLS')
    xls_file.to_csv(f'data/raw/cot/{file_name}.csv', index=False)